In [ ]:
import json
import simulate
import plotting

### First load the data

In [ ]:
FILE_PATH = "relation_MI_nudge_impact_1input.json"
with open(FILE_PATH, 'r') as f:
    data_dict = json.load(f)

In [ ]:
plot_range, mean, std, batches_std = simulate.find_mean_std_mse(
    data_dict, batch_size = 1
)
plot_range = [float(item) for item in plot_range]
print(mean)
print(std)

In [ ]:
plotting.plot_mean_and_confidence(plot_range, mean, std, 
                                  "confidence_interval_title", "xlabel", "ylabel", "title")

In [ ]:
import numpy as np
a1 = np.array([1, 2])
a2 = np.array([3, 4])
a3 = np.array([5, 6])
np.mean(np.array([a1, a2, a3]), axis=0)